### Лекция 7. Исключения

https://en.cppreference.com/w/cpp/language/exceptions

https://en.cppreference.com/w/cpp/error

https://apprize.info/c/professional/13.html

<br />

##### зачем нужны исключения

Для обработки исключительных ситуаций

<br />

Как вариант - обработка ошибок. Есть как минимум три стратегии обработки ошибок:
* через коды возврата
* через исключения
* сделать вид, что ошибок не существует

Типичная реализация функции, когда ошибки обрабатываются через коды возврата:

In [ ]:
int get_youngest_student(std::string& student_name)
{
    int err_code = 0;
    
    // # вытащить всех студентов из БД (пример: проброс ошибки)
    std::vector<Student> students;
    err_code = fetch_students(students);
    if (err_code != ErrCode::OK)
        return err_code;
    
    // # найти самого молодого (пример: ручное управление)
    auto youngest_it = std::min_element(students.begin(),
                                        students.end(),
                                        [](const auto& lhs, const auto& rhs){
                                            return lhs.age < rhs.age;
                                        });
    if (youngest_it == students.end())
        return ErrCode::NoStudents;
    
    // # вытащить из базы его имя (пример: частичная обработка)
    err_code = fetch_student_name(youngest_it->id, student_name);
    if (err_code != ErrCode::OK)
        if (err_code == ErrCode::NoObject)
            return ErrCode::CorruptedDatabase;
        else
            return err_code;
    
    return ErrCode::OK;
}

Типичная реализация в случае использования исключений:

In [ ]:
std::string get_youngest_student()
{
    // # вытащить всех студентов из БД (пример: проброс ошибки)
    std::vector<Student> students = fetch_students();
    
    // # найти самого молодого (пример: ручное управление)
    auto youngest_it = std::min_element(students.begin(),
                                        students.end(),
                                        [](const auto& lhs, const auto& rhs){
                                            return lhs.age < rhs.age;
                                        });
    if (youngest_it == students.end())
        throw std::runtime_error("students set is empty");
    
    // # вытащить из базы его имя (пример: частичная обработка)
    try
    {
        return fetch_student_name(youngest_it->id);
    }
    catch(const MyDBExceptions::NoObjectException& exception)
    {
        throw MyDBExceptions::CorruptedDatabase();
    }
}

Типичная реализация в случае игнорирования ошибок

In [ ]:
std::string get_youngest_student()
{
    // # вытащить всех студентов из БД (пример: проброс ошибки)
    std::vector<Student> students = fetch_students();  // # не кидает исключений,
                                                       // # никак не узнать, что проблемы с доступом к базе
    
    // # найти самого молодого (пример: ручное управление)
    auto youngest_it = std::min_element(students.begin(),
                                        students.end(),
                                        [](const auto& lhs, const auto& rhs){
                                            return lhs.age < rhs.age;
                                        });
    if (youngest_it == students.end())
        return "UNK";  // # не отделить ситуацию, когда нет студентов в базе вообще
                       // # от ситуации, когда в базе имя UNK у студента
    
    // # вытащить из базы его имя (пример: частичная обработка)
    return fetch_student_name(youngest_it->id);  // # не кидает исключений
    // # не отделить ситуацию, когда в таблице имён пропущен студент
    // # от ситуации, когда студент есть с именем UNK
}

**На практике: разумный баланс между детализацией ошибок и сложностью программы.**

<br />

##### как бросать и как ловить исключения, размотка стека

В момент исключительной ситуации:

In [ ]:
double my_sqrt(double x)
{
    if (x < 0)
        throw std::invalid_argument("sqrt can not be calculated for negatives in terms of double");
    
    ...;
}

Далее начинается размотка стека и поиск соответствующего catch-блока:

(объяснить на примере, показать 2 ошибки в коде)

In [ ]:
double get_radius_of_first_polyline_point()
{
    auto* logger = new Logger();
    
    std::vector<Point> polyline = make_polyline();
    Point p = polyline.front();
    
    logger->log("point is " + std::to_string(p.x) + " " + std::to_string(p.y));
    
    double r = my_sqrt(p.x * p.x - p.y * p.y);  // # ooops
    
    logger->log("front radius is " + std::to_string(r));
    delete logger;
    
    return r;
}


void func()
{
    try
    {
        std::cout << get_radius_of_first_polyline_point() << std::endl;
    }
    catch (const std::exception& e)
    {
        std::cout << "unknown exception: " << e.what() << std::endl;
    }
    catch (const std::invalid_argument& e)
    {
        std::cout << "aren't we trying to calculate sqrt of negatve? " << e.what() << std::endl;
    }
    catch (...)  // # you should never do that
    {
        std::cout << "what?" << std::endl;
    }
}

<br />

##### noexcept

Если функция не бросает исключений, желательно пометить её `noexcept`.

Вызов такой функции будет чуть дешевле и объём бинарного файла чуть меньше (не нужно генерировать кода поддержки исключений).

Что будет если `noexcept` - функция попытается бросить исключение?

In [ ]:
int get_sum(const std::vector<int>& v) noexcept
{
    return std::reduce(v.begin(), v.end());
}


int get_min(const std::vector<int>& v) noexcept
{
    if (v.empty())
        throw std::invalid_argument("can not find minimum in empty sequence");
    
    return *std::min_element(v.begin(), v.end());
}

<br />

##### стандартные и собственные классы исключений

![](std_exception_hierarchy.jpg)

Зачем свои классы исключений?
* большая детализация ошибки
* возможность добавить информацию к классу исключения

Рекомендации:
* свои классы наследовать от `std::exception`
* если возможно - организовать свои исключения в иерархию (чтобы была возможность ловить общую ошибку библиотеки или более детальные)
* если возможно - предоставить информацию для анализа и восстановления

Рассмотрим пример - вы пишете свой читатель-писатель json-ов (зачем? их миллионы уже!)

In [ ]:
namespace myjson
{
    // # общий наследник исключений вашей библиотеки парсинга,
    // # чтобы пользователи могли просто отловить события
    // # "в этой билиотеке что-то пошло не так"
    class MyJsonException : public std::exception
    {};
    
    // # исключение для случая, когда при запросе к полю у объекта
    // # поле отсутствовало
    class FieldNotFound : public MyJsonException
    {
    public:
        FieldNotFound(std::string parent_name,
                      std::string field_name);

        // # можно дать обработчику исключения больше информации
        // # для более разумной обработки ситуации или хотя бы
        // # более подробного логирования проблемы
        const std::string& parent_name() const noexcept;
        const std::string& field_name() const noexcept;
        
    private:
        std::string parent_name_;
        std::string field_name_;
    };
    
    // # исключение для ошибок при парсинге json-строки
    class ParsingException : public MyJsonException
    {
    public:
        ParsingException(int symbol_ix);
        
        // # можно дать больше информации, на каком символе
        // # обломился парсинг строки
        int symbol_ix() const noexcept;
        
    private:
        int symbol_ix_;
    };
    
    // # исключение для ошибок при парсинге int-а - сужение |ParsingException|
    class IntegerOverflowOnParsing  : public ParsingException
    {
    };
    
    // #  и т.д.
    
}  // namespace myjson

Опять же, на практике выбирают баланс между детализацией и сложностью.

Чем глубже детализация, тем сложнее программа, но с определённого уровня пользы от большей детализации мало.

<br />

##### исключения в конструкторах и деструкторах

Почему исключения полезны в конструкторах?

Потому что у конструктора нет другого способа сообщить об ошибке!

In [ ]:
std::vector<int> v = {1, 2, 3, 4, 5};
// # нет другого (нормального) способа сообщить вызываемому коду,
// # что памяти не хватило и вектор не создан, только бросить
// # исключение

Пример:

In [ ]:
class M {
    ...;
};

class B {
    ...;
};

class D : public B
{
public:
    D()
    {
        throw std::runtime_error("error");
    }
    
private:
    M m_;
};

Какие конструкторы и деструкторы будут вызваны?

In [ ]:
try
{
    D d;
}
catch (const std::exception& e)
{
}

А если так?

In [ ]:
class M {
    ...;
};

class B {
    ...;
};

class D : public B
{
public:
    D() : D(0)
    {
        throw std::runtime_error("error");
    }
    
    D(int x) {}
    
private:
    M m_;
};

* У созданного объекта будет вызван деструктор
* Объект считается созданным, если отработал *хотя бы один* его конструктор

Что с исключениями из деструкторов?

In [ ]:
class D
{
public:
    D() {}
    ~D() {
        throw std::runtime_error("cannot free resource");
    }    
};

* Бросать исключения из деструкторов - "плохо"
* По умолчанию деструктор - noexcept (если нет специфических проблем с базовыми классами и членами)
* Если при размотке стека из деструктора объекта бросается исключение, программа завершается с std::terminate по стандарту: https://en.cppreference.com/w/cpp/language/destructor (раздел Exceptions) "you can not fail to fail"

__Упражение__: чтобы понять, почему деструктору нельзя кидать исключения, попробуйте на досуге представить, как корректно реализовать `resize` у `std::vector<T>`, если `~T` иногда кидает исключение

<br />

##### гарантии при работе с исключениями

* `nothrow` - функция не бросает искючений

```
int swap(int& x, int& y)
```


* `strong` - функция отрабатывает как транзакция: если из функции вылетает исключение, состояние программы откатывается на момент как до вызова функции.

```
std::vector::push_back
```

* ```basic``` - если из функции вылетает исключение, программа ещё корректна (инварианты сохранены). Может потребоваться очистка.


```
void write_to_csv(const char* filename)
{
    std::ofsteam ofs(filename);
    ofs << "id,name,age" << std::endl;
    
    std::vector<std::string> names = ...; // bad_alloc
    ofs << ...;
}
```

* `no exception guarantee` - если из функции вылетает исключение, молитесь

```
любой production код (история про обработку ошибок в файловых системах)
```

* `exception-neutral` - только для шаблонных компонент - пропускает сквозь себя все исключения, которые кидают шаблонные параметры

```
std::min_element(v.begin(), v.end())
```

<br />

##### стоимость исключений

Зависит от реализации.

"В интернете пишут, что" исключения в основных компиляторах реализованы по принципу (бесплатно, пока не вылетел exception, дорого, если вылетел). При этом при выбросе исключений формируются специальные exception frame-ы, осуществляется поиск handler-ов и cleanup-процедур по заранее сгенерированным компилятором таблицам.

Подробнее:
* https://stackoverflow.com/questions/13835817/are-exceptions-in-c-really-slow
* https://mortoray.com/2013/09/12/the-true-cost-of-zero-cost-exceptions/

При этом код обслуживания исключений тоже надо сгенерировать. Статья как в microsoft провели исследование сколько занимает код обслуживания механизма исключений (спойлер: для конкретной билиотеки в районе 26% - зависит от кол-ва исключений, кол-ва бросающих исключения функций, кол-ва вызовов throw, сложности объектов на стеке и т.д.) и как его сократили где-то в 2 раза:

https://devblogs.microsoft.com/cppblog/making-cpp-exception-handling-smaller-x64/

Доклад как устроены исключения на Windows:

https://www.youtube.com/watch?v=COEv2kq_Ht8

<br />

##### noexcept-move-операции

Пояснить на классическом примере `std::vector::push_back` каким образом объявление move-операций `noexcept` позволяет ускорить программу:

![](vector_noexcept.png)

<br />

##### правила хорошего тона при реализации исключений

* Деструкторы не должны бросать исключений. Можете помечать их `noexcept` или знать, что компилятор во многих случаях автоматически добавляет `noexcept` к деструкторам.
* Реализовывать и помечать move-операции как `noexcept`
* Реализовывать и помечать default constructor как `noexcept` (cppcoreguildelines для скорости)
* `noexcept` everything you can!
* Цитата с cppcoreguidelines: `If you know that your application code cannot respond to an allocation failure, it may be appropriate to add noexcept even on functions that allocate.` Объяснить её смысл про восстановление после ошибки и почему "нелогичность" здесь полезна.
* Пользовательские классы исключений наследовать от `std::exception` или его подклассов
* Ловить исключений по const-ссылкам
* `throw;` вместо `throw e;` из catch-блока, когда нужен rethrow
* Исключения являются частью контракта (или спецификации) функции! Желательно их протестировать.
* Использовать исключения для исключительных ситуаций, а не для естественного потока выполнения.
  * Плохой код:
    * исключение чтобы прервать цикл
    * исключение чтобы сделать особое возвращаемое значение из функции
  * Приемлемо:
    * исключение чтобы сообщить об ошибке
    * исключение чтобы сообщить о нарушении контракта (объяснить, почему это не лучший вариант использования исключений)
* Исключения служат для того чтобы восстановиться после ошибки и продолжить работу:
  * пропустить некритичные действия. Пример:
    * отобразить телефон организации в информационном листе)
  * fallback с восстановлением или откатом:
    * memory-consuming алгоритмы
    * null-screen-ы (search failed)
    * message box-ы об ошибке (не удалось открыть документ в msword)
  * красиво умереть на критических ошибках:
    * memory allocation on game start
    * некорректная команда в текстовом интерпретаторе
* Глобальный try-catch:

    ```
    int main() {
        try {
            ...
        } catch(const std::exception& e) {
            std::cout << "Until your last breath!\n";
            std::cout << "ERROR: " << e.what() << std::endl;
        }
    }
    ```